
https://arxiv.org/pdf/1805.12307.pdf 적용 예정.
2020.04.21 -> preprocessing까지

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
import pandas as pd

In [0]:
from tqdm import tqdm_notebook

In [0]:
import os
os.chdir('/content/gdrive/My Drive/Colab Notebooks/toxic/jigsaw-toxic-comment-classification-challenge')

In [0]:
train = pd.read_csv('train.csv.zip',compression='zip')
test = pd.read_csv('test.csv.zip', compression='zip')
test_label = pd.read_csv('test_labels.csv.zip',compression='zip')

In [0]:
y = train.iloc[:,2:].values

In [0]:
train.iloc[:,2:].sum(axis=0)

toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from collections import Counter
from nltk.corpus import stopwords
import os, pickle, re
from nltk.tokenize import word_tokenize  
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Concatenate

stop_eng = stopwords.words('english')

def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')

class makeData:
    def __init__(self, maxlen, embed_dim, minCount, mode='embedLayer'):
        self.maxlen = maxlen
        self.embed_dim = embed_dim
        self.minCount = minCount
        if mode not in ['embedLayer','FastText']:
            raise ValueError("Can not understand %s mode"%(mode))
        self.mode = mode
        self.preprocessed_train_title = 'preprocessed_train_'+str(maxlen)+'_'+str(embed_dim)+'_'+str(minCount)+'.pkl'
        self.tokenized_train_title = 'tokenized_train_'+str(maxlen)+'_'+str(embed_dim)+'_'+str(minCount)+'.pkl'
        self.preprocessed_test_title = 'preprocessed_test_'+str(maxlen)+'_'+str(embed_dim)+'_'+str(minCount)+'.pkl'
        self.tokenized_test_title = 'tokenized_test_'+str(maxlen)+'_'+str(embed_dim)+'_'+str(minCount)+'.pkl'
        self.word_list_name = 'word_list_'+mode+'_'+str(minCount)+'.pkl'
        self.word_list = None
    
    def run(self,train,test):
        df_train = self.fit(train,True)
        df_test = self.fit(test,False)
        return df_train, df_test
    
    def fit(self,df,is_train=False):#문장들만 넣어주세요
        #전처리 끝난 데이터 있나요?
        if self.preprocessed_train_title in os.listdir():
            df =load_data(self.preprocessed_train_title)
            return df
        else:
            #토큰화 끝난 데이터 있나요?
            if self.tokenized_train_title in os.listdir():
                df = load_data(self.tokenized_train_title)
            else:
                #tokenizer fit
                if is_train == True:
                    #토큰화 + 특수문자제거
                    tokenizer = Tokenizer()
                    tokenizer.fit_on_texts(df)
                    #-------------------------
                    #tokenizer는 dict 기반으로 단어->숫자 변환
                    #word_index의 값을 삭제해 mincount, 불용어 처리
                    #-------------------------
                    #불용어 처리
                    for i in stop_eng:
                        tokenizer.word_index.pop(i,None)                    
                    #minCount이하의 출현->삭제목록
                    lessCount = dict(filter(lambda x:x[1]<5,tokenizer.word_counts.items()))
                    for w in lessCount.keys():
                        tokenizer.word_index.pop(w,None)
                    self.tokenizer = tokenizer               
                #test데이터는 train 데이터를 통해 토큰화 목록에 있는 친구들만 토큰화    
                df=self.tokenizer.texts_to_sequences(df)
            #토큰화 종료 -> len 맞춤
            df = pad_sequences(df, maxlen=self.maxlen)
            return df        
    #이전버전
    '''
    def preprocess(self, df, is_train):
        if is_train :
        #모델에 임베딩 레이어 사용할 때
            if self.word_list is None:
                print('start to make valid word list')
                if self.mode == 'embedLayer':
                    counter = []
                    for i in df:
                        counter.extend(i)
                    counter = Counter(counter)
                    self.word_list = dict(list(filter(lambda x:x[1]>self.minCount, counter.items())))
                    save_data(self.word_list_name,self.word_list)

                #FastText로 임베딩된 워드 사용할 때
                ## 0422 pretrainned Fast text 사용
                elif self.mode == 'FastText':
                    #name = 'fasttext_'+str(self.maxlen)+'_'+str(self.embed_dim)+'.vec'
                    name = 'crawl-300d-2M.vec'
                    if name in os.listdir():
                        print("apply %s"%(name))
                        #self.ft_model = FastText.load(name)
                        self.word_list = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open('crawl-300d-2M.vec'))                    
                    else:
                        self.ft_model = FastText(train, size=embed_dim, window=5, min_count=self.minCount, workers=4, sg=1)
                        self.ft_model.save(name)
                        save_data(self.word_list_name,self.word_list)
                        self.word_list = dict(self.ft_model.wv.vocab.items())
        #----------
        #word_list에 없는 단어들 버림
        #1 epoch에 30분->10분 단축, accuracy 0.01상승
        print('start to filter words')
        for i, sentence in enumerate(df):
            df[i] = list(filter(lambda x: x in self.word_list, sentence))
        #이제는 mincount 이상의 빈도로 출현한 단어만 적용됨
        #-----------
        #문장의 단어 개수 통일        
        print('start to fit shape')
        if self.mode == 'embedLayer':
            if is_train == True:
                tokenizer = Tokenizer()
                tokenizer.fit_on_texts(df)
                self.el_tokenizer = tokenizer
                
                df = self.el_tokenizer.texts_to_sequences(df)
                df = pad_sequences(df, maxlen=self.maxlen)
                return df
            else:                
                df = self.el_tokenizer.texts_to_sequences(df)
                df = pad_sequences(df, maxlen=self.maxlen)
                return df
        elif self.mode == 'FastText':
            outlayer = np.zeros((len(df),self.maxlen,self.embed_dim))
            for i, sentence in enumerate(tqdm_notebook(df)):
                temp = np.zeros((self.maxlen,self.embed_dim))
                index = self.maxlen-len(sentence) if len(sentence) < self.maxlen else 0
                for word in sentence:
                    if index == self.maxlen:
                        break
                    temp[index] = self.word_list[word]
                    index+=1
                outlayer[i] = temp
            return outlayer        
'''

In [0]:
class Attention(tf.keras.Model):
	def __init__(self, units):
		super(Attention, self).__init__()
		self.W1 = tf.keras.layers.Dense(units)
		self.W2 = tf.keras.layers.Dense(units)
		self.V = tf.keras.layers.Dense(1)

	def call(self, features, hidden):
		#bidirectional LSTM 통해 나온 encoded sequence(x)와 hidden state(state_h)를 더해주기 위해 hidden_staet의 shape를 조절함
		hidden_with_time_axis = tf.expand_dims(hidden, 1) #지금 단어 이전/이후 정보 있는 hidden state
		''' 
		# score shape == (batch_size, max_length, 1)
		# we get 1 at the last axis because we are applying score to self.V
		# the shape of the tensor before applying self.V is (batch_size, max_length, units)
		'''
		score = tf.nn.tanh(
			self.W1(features) + self.W2(hidden_with_time_axis)) #encoded sequence와 hidden state 더해주고 tanh로 activation
		# attention_weights shape == (batch_size, max_length, 1)
		attention_weights = tf.nn.softmax(self.V(score), axis=1) #softmax에 넘겨줘서 이를 0부터 1까지의 값을 가지는 attention score화
		  
		# context_vector shape after sum == (batch_size, hidden_size)
		context_vector = attention_weights * features #각각의 단어의 가중치(attention score)와 encoded sequences를 곱해줘서 context vector에 저장
		context_vector = tf.reduce_sum(context_vector, axis=1)
		return context_vector, attention_weights

In [0]:
import sys,  re, csv, codecs
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
def predict_model_FastText(df_train,y,df_test):
    inp = Input(shape=(maxlen,embed_dim, ))
    x = LSTM(60, return_sequences=True,name='lstm_layer')(inp)
    x = GlobalMaxPool1D()(x)
    x = Dense(200, activation="relu")(x)
    x = Dense(6, activation="sigmoid")(x)

    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
    batch_size = 256
    epochs = 2
    model.fit(df_train,y,batch_size=batch_size, epochs=epochs, validation_split=0.1)
    model.save('model'+str(maxlen)+str(embed_dim)+'.h5')
    t = model.predict(df_test, batch_size=batch_size)
    sub = pd.read_csv('sample_submission.csv.zip',compression='zip')
    subm = pd.DataFrame(t,columns=sub.columns[1:])
    subm['id'] = sub['id']
    subm.to_csv('sub4.csv',index=False)
    return model
    
def predict_model(df_train,y,df_test):
    max_feature = max(list(map(lambda x: max(x),df_train)))*2
    inp = Input(shape=(maxlen,))
    x = Embedding(max_feature,embed_dim)(inp)
    x = Bidirectional(LSTM(60, return_sequences=True,name='lstm_layer'))(x)
    #Getting our Bidirectional LSTM output
    (x, forward_h, forward_c, backward_h, backward_c) = Bidirectional(LSTM(60, return_sequences=True, return_state=True))(x)
    #forward, backward hidden state 합쳐주는 과정
    state_h = Concatenate()([forward_h, backward_h])
    state_c = Concatenate()([forward_c, backward_c])
    #Attention Layer 적용
    context_vector, attention_weights = Attention(10)(x, state_h)
    x = Dense(50, activation = 'relu')(context_vector) #context vector를 다시 dense로 펼쳐줌
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)

    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
    batch_size = 32
    epochs = 2
    model.fit(df_train,y,batch_size=batch_size, epochs=epochs, validation_split=0.1,)
    # model.save('model'+str(maxlen)+str(embed_dim)+'.h5')
    t = model.predict(df_test, batch_size=batch_size)
    sub = pd.read_csv('sample_submission.csv.zip',compression='zip')
    subm = pd.DataFrame(t,columns=sub.columns[1:])
    subm['id'] = sub['id']
    
    subm.to_csv('mySubmission.csv',index=False)
    return model    

In [0]:
maxlen,embed_dim = 120, 200
dataBuilder = makeData(maxlen, embed_dim, 5)

df_train, df_test = dataBuilder.run(train['comment_text'],test['comment_text'])

In [0]:
model = predict_model(df_train,y,df_test)

Epoch 1/2
4488/4488 [==============================] - 632s 141ms/step - loss: 0.0637 - accuracy: 0.9599 - val_loss: 0.0502 - val_accuracy: 0.9940
Epoch 2/2
4488/4488 [==============================] - 632s 141ms/step - loss: 0.0434 - accuracy: 0.9916 - val_loss: 0.0489 - val_accuracy: 0.9940
